In [37]:
raw_text_folder = "/Users/leduc/py_source/spell_correction/corpus.viwiki/viwiki"

In [13]:
def read_raw_text(filepath):
    with open(filepath, 'r', encoding='utf-8') as file:
        return file.read()
def remove_special_characters(text):
    # Remove special characters 
    char_list = ['\n', '\t',  '!', '#', '$', '%', '&',  '(', ')', '*', '+', ',', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~']
    for char in char_list:
        text = text.replace(char, ' ')
    return text


In [71]:
import os
from tqdm import tqdm
from pyvi import ViTokenizer, ViPosTagger
# count the frequency of words in the corpus
import re
import collections
word_list = []
for file in tqdm(os.listdir(raw_text_folder)):
    if file.endswith('.txt'):
        filepath = os.path.join(raw_text_folder, file)
        text = read_raw_text(filepath)
        text = remove_special_characters(text.lower())
        text = ViTokenizer.tokenize(text)
        
        word_list.extend(re.findall(r'\b\w+\b', text))
         
            

# create frequency dictionary
frequency_dict = collections.Counter(word_list)

100%|██████████| 7878/7878 [01:14<00:00, 105.26it/s]


In [72]:
frequency_dict

Counter({'và': 238932,
         'của': 202582,
         'các': 170656,
         'là': 170422,
         'được': 128383,
         'trong': 125926,
         'năm': 112317,
         'một': 111223,
         'có': 103742,
         'người': 87104,
         'với': 79646,
         'đã': 71674,
         'cho': 70241,
         'những': 67273,
         'ở': 63335,
         'từ': 57298,
         'vào': 56419,
         'tháng': 52669,
         'không': 50119,
         'này': 50038,
         'đến': 47328,
         'tại': 45989,
         'khi': 45306,
         'về': 43112,
         'nam': 42392,
         'như': 41305,
         'đó': 40898,
         'để': 39641,
         'trên': 37802,
         'sau': 36932,
         'sự': 35232,
         'nhiều': 34737,
         'ra': 33887,
         'cũng': 33790,
         'theo': 31945,
         'ông': 31056,
         'ngày': 29568,
         '1': 29008,
         'nhà': 28534,
         'bị': 28132,
         'nước': 27364,
         'nhất': 26755,
         'anh': 26324

In [108]:
# remove words with frequency less than 100
frequency_dict = {word: freq for word, freq in frequency_dict.items() if freq >= 1000}


In [72]:
# save to vi_dict.txt
output_file = 'vi_dict.txt'


In [ ]:
with open(output_file, 'w', encoding='utf-8') as f:
    for word, freq in frequency_dict.items():
        f.write(f"{word} {freq}\n")

In [73]:
with open(output_file, 'r', encoding='utf-8') as f:
    lines = f.readlines()
    for line in lines:
        print(line.strip())

lãnh_thổ 4911
lương_cường 100000000
hồ_chí_minh 20000
châu 18733
nam 42392
cực 1627
thuộc 14354
úc 1939
là 170422
một 111223
phần 10784
của 202582
vương_quốc 3967
anh 26324
từng 6490
chủ_quyền 1231
khu_vực 11140
này 50038
và 238932
đặt 6611
dưới 9700
quyền 10088
điều_hành 2220
chung 4138
năm 112317
đây 14077
lớn 18996
nhất 26755
ở 63335
bị 28132
từ 57298
khi 45306
hiệp_ước 2051
có 103742
điều 7096
1 29008
rõ 1786
không 50119
công_nhận 2639
tranh_cãi 1139
hay 17308
bất_cứ 1062
yêu_cầu 2501
nào 6126
sẽ 14292
được 128383
còn 23514
hầu_hết 3154
các 170656
quốc_gia 18356
trên 37802
thế_giới 13634
những 67273
phân_chia 1254
hành_chính 3384
chia 4079
thành 18842
9 10497
quận 5111
bao_gồm 8381
đối_với 6265
chỉ 19036
bốn 2880
gồm 8758
new 4925
pháp 15548
na 1375
uy 1096
trong 125926
đáng 1618
chú_ý 1378
nhật 13241
bản 12232
cũng 33790
nơi 9645
mà 17055
thực_hiện 5619
việc 22900
chú_thích 2743
liên_kết 6084
tiếng 18560
of 14458
the 18946
nhà 28534
phát_triển 13030
video 1345
game 1423
hoa 20454


In [85]:
from symspellpy import SymSpell, Verbosity

In [86]:
sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)

In [87]:
new_list = []
with open('name_dict.txt', 'r', encoding='utf-8') as f:
    for line in f:
        term = line.strip()
        if not term:
            continue
        term = term.lower()
        # term = term.replace(' ', '_')  # Replace spaces with underscores
        newline = term + ' 10000'  # Assuming a default count for names
        new_list.append(newline)    
with open('new_name_dict.txt', 'w', encoding='utf-8') as f:
    for item in new_list:
        f.write(f"{item}\n")

In [ ]:
dictionary_path = "new_name_dict.txt"
sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1)

True

In [91]:
sym_spell.create_dictionary_entry("SymSpell", 1)

input_term = "thủ tướng nguyễn minh chính"

suggestions = sym_spell.lookup(input_term, Verbosity.CLOSEST, max_edit_distance=2)

for suggestion in suggestions:
    print(suggestion.term, suggestion.distance, suggestion.count)

In [79]:
def check_and_correct_word(word):
    suggestions = sym_spell.lookup(word, Verbosity.CLOSEST, max_edit_distance=10)
    if suggestions:
        if word != suggestions[0].term:
            return word, "incorrect!", "correct_version: " + suggestions[0].term
        else:
            return word, "correct!"

    return word, "not_found"  # Return the original word if no suggestions found

In [80]:

from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline




def get_raw_ner(model_path:str, query: str):
    tokenizer = AutoTokenizer.from_pretrained(f"{model_path}")
    model = AutoModelForTokenClassification.from_pretrained(f"{model_path}")
    nlp = pipeline("ner", model=model, tokenizer=tokenizer)
    ner_results = nlp(query)
    return ner_results
def extract_entities(ner_results):
    entities = []
    current_entity = ""
    current_type = None


    for item in ner_results:
        ent_type = item['entity'].split('-')[-1]
        if item['entity'].startswith('B-'):
            if current_entity:
                entities.append((current_entity.strip(), current_type))
            current_entity = item['word']
            current_type = ent_type
        elif item['entity'].startswith('I-') and current_type == ent_type:
            current_entity += " " + item['word']
        else:
            if current_entity:
                entities.append((current_entity.strip(), current_type))
            current_entity = ""
            current_type = None

    if current_entity:
        entities.append((current_entity.strip(), current_type))

    return entities


In [81]:
text = 'Tổng bí thư Tôn Lâm có bài phát biểu tại hội nghị'
model_path = "NlpHUST/ner-vietnamese-electra-base"


In [82]:

print(check_and_correct_word("thủ_tướng_thái_lan"))
print(check_and_correct_word("tôn_lâm"))
print(check_and_correct_word("thủ_tướng_nguyễn_minh_chính"))
print(check_and_correct_word("thủ_tướng_phạm_minh_chính"))
print(check_and_correct_word("tổng_thống_hồ_chí_minh"))
print(check_and_correct_word("tổng_bí_thư_tôn_lâm"))
print(check_and_correct_word("tổng_bí_thư_tô_lâm"))
print(check_and_correct_word("tổng_bí_thư_tô_lân"))
print(check_and_correct_word("lương_tam_quang"))
print(check_and_correct_word("lươn_tam_quang"))





('thủ_tướng_thái_lan', 'incorrect!', 'correct_version: thủ_tướng')
('tôn_lâm', 'incorrect!', 'correct_version: tô_lâm')
('thủ_tướng_nguyễn_minh_chính', 'incorrect!', 'correct_version: thủ_tướng_phạm_minh_chính')
('thủ_tướng_phạm_minh_chính', 'correct!')
('tổng_thống_hồ_chí_minh', 'incorrect!', 'correct_version: chủ_tịch_hồ_chí_minh')
('tổng_bí_thư_tôn_lâm', 'incorrect!', 'correct_version: tổng_bí_thư_tô_lâm')
('tổng_bí_thư_tô_lâm', 'correct!')
('tổng_bí_thư_tô_lân', 'incorrect!', 'correct_version: tổng_bí_thư_tô_lâm')
('lương_tam_quang', 'correct!')
('lươn_tam_quang', 'incorrect!', 'correct_version: lương_tam_quang')


In [70]:
from pyvi import ViTokenizer, ViPosTagger
import logging 
# off logging
logging.disable(logging.CRITICAL)
incorrect_sentence = "Tổng bí thư Tôn Lâm và thủ tướng nguyễn Minh Chính có bài phát biểu tại hội nghị chuyển đổi số quốc gia của Việt Nam"
_input = ViTokenizer.tokenize(incorrect_sentence)
print("token:" ,_input)
ner_results = get_raw_ner(model_path, incorrect_sentence)
# print(ner_results)
entities = extract_entities(ner_results)
for entity, ent_type in entities:
    entity = entity.replace(' ', '_')  # Replace spaces with underscores
    entity = entity.lower()  # Convert to lowercase
    # if before the entity is the title "thủ tướng" or "tổng bí thư", then it is a name
    
    print(f"Entity: {entity}, Type: {ent_type}")
    print(check_and_correct_word(entity))

token: Tổng_bí_thư Tôn_Lâm và thủ_tướng nguyễn Minh Chính có bài phát_biểu tại hội_nghị chuyển_đổi số quốc_gia của Việt_Nam
Entity: tôn_lâm, Type: PERSON
('tôn_lâm', 'incorrect!', 'correct_version: tô_lâm')
Entity: nguyễn_minh_chính, Type: PERSON
('nguyễn_minh_chính', 'incorrect!', 'correct_version: nguyễn_mạnh_cường')
Entity: việt_nam, Type: LOCATION
('việt_nam', 'incorrect!', 'correct_version: tô_lâm')


In [53]:
for word in _input.split():
    word, status, *correct_version = check_and_correct_word(word)
    if status == "incorrect!":
        print(f"Word: {word}, Status: {status}, Correct Version: {correct_version[0]}")
    elif status == "correct!":
        print(f"Word: {word}, Status: {status}")
    else:
        print(f"Word: {word}, Status: {status}")  # not_found case

Word: Tổng_bí_thư, Status: incorrect!, Correct Version: correct_version: tổng_bí_thư
Word: Tôn_Lâm, Status: incorrect!, Correct Version: correct_version: tô_lâm
Word: có, Status: incorrect!, Correct Version: correct_version: tô_lâm
Word: bài, Status: incorrect!, Correct Version: correct_version: tô_lâm
Word: phát_biểu, Status: incorrect!, Correct Version: correct_version: thủ_tướng
Word: tại, Status: incorrect!, Correct Version: correct_version: tô_lâm
Word: hội_nghị, Status: incorrect!, Correct Version: correct_version: hà_thị_nga
Word: chuyển, Status: incorrect!, Correct Version: correct_version: tô_lâm
Word: đổ, Status: incorrect!, Correct Version: correct_version: tô_lâm
Word: số, Status: incorrect!, Correct Version: correct_version: tô_lâm
Word: quốc_gia, Status: incorrect!, Correct Version: correct_version: lê_quốc_minh
Word: của, Status: incorrect!, Correct Version: correct_version: tô_lâm
Word: Việt_Nam, Status: incorrect!, Correct Version: correct_version: tô_lâm
